In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
tqdm.pandas()


#libraries required for NetworkX
import networkx as nx

In [2]:
Recommender_Graph = nx.Graph()

In [3]:
movie_v2 = pd.read_csv("movies_wise_summary_V2.csv")
ratings= pd.read_csv("ratings_v2.csv")

In [4]:
ratings_v2 = ratings.copy()

In [5]:
ratings_v2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000209 entries, 0 to 1000208
Data columns (total 11 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   userId      1000209 non-null  int64 
 1   movieId     1000209 non-null  int64 
 2   rating      1000209 non-null  int64 
 3   title       1000209 non-null  object
 4   genres      998034 non-null   object
 5   year        1000209 non-null  int64 
 6   meta_data   998034 non-null   object
 7   gender      1000209 non-null  object
 8   age         1000209 non-null  int64 
 9   occupation  1000209 non-null  object
 10  zipcode     1000209 non-null  object
dtypes: int64(5), object(6)
memory usage: 83.9+ MB


## Creating Nodes for the Recommender Graph

In [18]:
ratings_v2[["userId","movieId"]] = ratings_v2[['userId', 'movieId']].astype(str)

In [245]:
ratings_v2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000209 entries, 0 to 1000208
Data columns (total 11 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   userId      1000209 non-null  object
 1   movieId     1000209 non-null  object
 2   rating      1000209 non-null  int64 
 3   title       1000209 non-null  object
 4   genres      998034 non-null   object
 5   year        1000209 non-null  int64 
 6   meta_data   998034 non-null   object
 7   gender      1000209 non-null  object
 8   age         1000209 non-null  int64 
 9   occupation  1000209 non-null  object
 10  zipcode     1000209 non-null  object
dtypes: int64(3), object(8)
memory usage: 83.9+ MB


In [19]:
ratings_v2["userId"] = ratings_v2["userId"]+"$$u"  #u subscript for user
ratings_v2["movieId"] = ratings_v2["movieId"]+"$$m"  #u subscript for user

In [247]:
#I think movie related nodes should be made from movie metadata and user related information nodes should be made separately
ratings_v2.head(2)

,userId,movieId,rating,title,genres,year,meta_data,gender,age,occupation,zipcode
0,1$$u,1$$m,5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995,"{'adult': False, 'backdrop_path': '/3Rfvhy1Nl6...",F,1,K-12 student,48067
1,1$$u,2692$$m,4,Run Lola Run (Lola rennt) (1998),Action|Crime,1998,"{'adult': False, 'backdrop_path': '/lhsrT0SbPa...",F,1,K-12 student,48067


In [248]:
Recommender_Graph

In [249]:
Recommender_Graph= nx.from_pandas_edgelist(ratings_v2, source='userId', target='movieId', edge_attr="rating")

In [250]:
print(Recommender_Graph.has_edge('1$$u','1$$m'))

True


In [251]:
Recommender_Graph.number_of_nodes()
nx.write_gpickle(Recommender_Graph, "Recommender_graph.gpickle")

In [252]:
Recommender_Graph_v2 = nx.read_gpickle("Recommender_graph.gpickle")

In [253]:
Recommender_Graph_v2.number_of_edges()

1000209

In [ ]:
#adding movie_summary nodes and edges
#adding_movieid & Genre Nodes edges

In [254]:
movie_v2["movieId"] = movie_v2["movieId"].astype(str)

In [255]:
movie_v2["movieId"] = movie_v2["movieId"] + "$$m"

In [256]:
movie_genre_edges = movie_v2[["movieId","genres"]]

In [257]:
type(movie_genre_edges["genres"])

pandas.core.series.Series

In [258]:
movie_genre_edges["genres"]= movie_genre_edges["genres"].apply(lambda x: list(x.split("|")))

<ipython-input-258-000482facf19>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie_genre_edges["genres"]= movie_genre_edges["genres"].apply(lambda x: list(x.split("|")))


In [259]:
#explode works on a list of items
movie_genre_edges = movie_genre_edges.explode(column="genres",ignore_index=False)

In [260]:
movie_genre_edges.reset_index(inplace=True)

In [261]:
movie_genre_edges.drop('index',axis=1,inplace=True)

In [262]:
movie_genre_edges["genres"]= movie_genre_edges["genres"] + "$$genres"

In [263]:
#the below code makes the list of all the columns and then we explode on the basis of list (explode is always applied on a list)
movie_genre_edges['combined']= movie_genre_edges.values.tolist()

In [264]:
type(movie_genre_edges["combined"])

pandas.core.series.Series

In [265]:
movie_genre_edges["combined_tuple"]=movie_genre_edges["combined"].apply(lambda x: tuple(x))

In [266]:
movie_genre_edges_list = list(movie_genre_edges["combined_tuple"])

In [267]:
Recommender_Graph_v2.add_edges_from(movie_genre_edges_list,weight=4)

In [268]:
Recommender_Graph_v2.number_of_nodes()

9765

In [269]:
nx.write_gpickle(Recommender_Graph_v2, "Recommender_graph.gpickle")

In [270]:
#adding movie & year edges

In [271]:
movie_and_year = movie_v2[["movieId","year"]]

In [272]:
movie_and_year[["movieId","year"]] = movie_and_year[["movieId","year"]].astype(str)
movie_and_year["year"] = movie_and_year["year"] + "$$year"

C:\Users\hp\anaconda3\lib\site-packages\pandas\core\frame.py:3191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
<ipython-input-272-cf6700df1024>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie_and_year["year"] = movie_and_year["year"] + "$$year"


In [273]:
movie_and_year['combined']= movie_and_year.values.tolist()

<ipython-input-273-a2d0138b27b9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie_and_year['combined']= movie_and_year.values.tolist()


In [274]:
movie_and_year["combined_tuple"]=movie_and_year["combined"].apply(lambda x: tuple(x))

<ipython-input-274-cd190f023d7b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie_and_year["combined_tuple"]=movie_and_year["combined"].apply(lambda x: tuple(x))


In [275]:
#for using add_edges_from we need a list of tuples
Recommender_Graph_v2.add_edges_from(list(movie_and_year["combined_tuple"]),weight = 4)

In [276]:
Recommender_Graph_v2.number_of_nodes()

9846

In [277]:
nx.write_gpickle(Recommender_Graph_v2, "Recommender_graph_year+genre.gpickle")

In [278]:
movie_and_lang = movie_v2[["movieId","original_language"]]

In [280]:
movie_and_lang["original_language"] = movie_and_lang["original_language"] + "$$original_language"

<ipython-input-280-7689e7e03909>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie_and_lang["original_language"] = movie_and_lang["original_language"] + "$$original_language"


In [281]:
movie_and_lang['combined']= movie_and_lang.values.tolist()

<ipython-input-281-57bd1cf2a4f8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie_and_lang['combined']= movie_and_lang.values.tolist()


In [282]:
movie_and_lang["combined_tuple"]=movie_and_lang["combined"].apply(lambda x: tuple(x))

<ipython-input-282-ab61cd029184>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie_and_lang["combined_tuple"]=movie_and_lang["combined"].apply(lambda x: tuple(x))


In [283]:
Recommender_Graph_v2.add_edges_from(list(movie_and_lang["combined_tuple"]),weight = 4)

In [284]:
production_countries = movie_v2[["movieId","production_countries"]].copy()

In [285]:
production_countries.shape

(3685, 2)

In [286]:
production_countries.dropna(axis=0,inplace=True)

In [287]:
production_countries

,movieId,production_countries
0,1$$m,"[{'iso_3166_1': 'US', 'name': 'United States o..."
1,10$$m,"[{'iso_3166_1': 'GB', 'name': 'United Kingdom'..."
2,100$$m,"[{'iso_3166_1': 'US', 'name': 'United States o..."
3,1000$$m,"[{'iso_3166_1': 'US', 'name': 'United States o..."
4,1002$$m,[]
...,...,...
3680,994$$m,"[{'iso_3166_1': 'US', 'name': 'United States o..."
3681,996$$m,"[{'iso_3166_1': 'US', 'name': 'United States o..."
3682,997$$m,"[{'iso_3166_1': 'US', 'name': 'United States o..."
3683,998$$m,"[{'iso_3166_1': 'US', 'name': 'United States o..."


In [288]:
production_countries["without_string"] = production_countries["production_countries"].apply(lambda x: eval(x))

In [289]:
production_countries = production_countries.explode(column = "without_string",ignore_index=False)

In [290]:
production_countries["without_string"].isna().sum()
#125 entries are missing so dropping them off
print(production_countries.shape)
production_countries.dropna(axis=0,inplace=True)
print(production_countries.shape)

(4681, 3)
(4556, 3)


In [291]:
production_countries.reset_index(inplace=True)

In [292]:
production_countries["without_string"].apply(lambda x: pd.DataFrame(x,index=[0])["name"])

,0
0,United States of America
1,United Kingdom
2,United States of America
3,United States of America
4,United States of America
...,...
4551,United States of America
4552,United States of America
4553,United States of America
4554,United States of America


In [293]:
production_countries["final_production_country"] = production_countries["without_string"].apply(lambda x: pd.DataFrame(x,index=[0])["name"])

In [294]:
production_countries

,index,movieId,production_countries,without_string,final_production_country
0,0,1$$m,"[{'iso_3166_1': 'US', 'name': 'United States o...","{'iso_3166_1': 'US', 'name': 'United States of...",United States of America
1,1,10$$m,"[{'iso_3166_1': 'GB', 'name': 'United Kingdom'...","{'iso_3166_1': 'GB', 'name': 'United Kingdom'}",United Kingdom
2,1,10$$m,"[{'iso_3166_1': 'GB', 'name': 'United Kingdom'...","{'iso_3166_1': 'US', 'name': 'United States of...",United States of America
3,2,100$$m,"[{'iso_3166_1': 'US', 'name': 'United States o...","{'iso_3166_1': 'US', 'name': 'United States of...",United States of America
4,3,1000$$m,"[{'iso_3166_1': 'US', 'name': 'United States o...","{'iso_3166_1': 'US', 'name': 'United States of...",United States of America
...,...,...,...,...,...
4551,3680,994$$m,"[{'iso_3166_1': 'US', 'name': 'United States o...","{'iso_3166_1': 'US', 'name': 'United States of...",United States of America
4552,3681,996$$m,"[{'iso_3166_1': 'US', 'name': 'United States o...","{'iso_3166_1': 'US', 'name': 'United States of...",United States of America
4553,3682,997$$m,"[{'iso_3166_1': 'US', 'name': 'United States o...","{'iso_3166_1': 'US', 'name': 'United States of...",United States of America
4554,3683,998$$m,"[{'iso_3166_1': 'US', 'name': 'United States o...","{'iso_3166_1': 'US', 'name': 'United States of...",United States of America


In [295]:
movie_and_prod_country = production_countries[["movieId","final_production_country"]].copy()

In [296]:
movie_and_prod_country["final_production_country"] = movie_and_prod_country["final_production_country"]+"$$production_country"

In [297]:
movie_and_prod_country.head(2)
movie_and_prod_country['combined']= movie_and_prod_country.values.tolist()
movie_and_prod_country["combined_tuple"]=movie_and_prod_country["combined"].apply(lambda x: tuple(x))
Recommender_Graph_v2.add_edges_from(list(movie_and_prod_country["combined_tuple"]),weight = 4)

In [298]:
Recommender_Graph_v2.number_of_nodes()

9945

In [299]:
nx.write_gpickle(Recommender_Graph_v2, "Recommender_graph_year+genre+prod_country.gpickle")

In [6]:
Recommender_Graph_v2 = nx.read_gpickle("Recommender_graph_year+genre+prod_country.gpickle")

In [7]:
ratings_v2.head(2)

,userId,movieId,rating,title,genres,year,meta_data,gender,age,occupation,zipcode
0,1$$u,1$$m,5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995,"{'adult': False, 'backdrop_path': '/3Rfvhy1Nl6...",F,1,K-12 student,48067
1,1$$u,2692$$m,4,Run Lola Run (Lola rennt) (1998),Action|Crime,1998,"{'adult': False, 'backdrop_path': '/lhsrT0SbPa...",F,1,K-12 student,48067


In [27]:
movie_v2["movieId"] =movie_v2["movieId"].astype(str)
movie_v2["movieId"] = movie_v2["movieId"] +"$$m"
movie_v2.to_csv('movies_wise_summary_V2_modified_movie_id_$$m.csv',index=False)

In [28]:
movie_popularity = movie_v2[["movieId","popularity"]].copy()

In [29]:
movie_popularity.head(2)
print(movie_popularity.info())
#popularity column is much of it is populated
#1st converting the popularity column into a whole no

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3685 entries, 0 to 3684
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   movieId     3685 non-null   object 
 1   popularity  3666 non-null   float64
dtypes: float64(1), object(1)
memory usage: 57.7+ KB
None


In [30]:
movie_popularity.dropna(subset=['popularity'],axis=0,inplace=True)

In [31]:
movie_popularity["popularity_ceiling"] = movie_popularity["popularity"].apply(lambda x: math.ceil(x))

In [32]:
movie_popularity = movie_popularity[["movieId","popularity_ceiling"]]

In [33]:
movie_popularity["popularity_ceiling"] = movie_popularity["popularity_ceiling"].astype(str)

In [34]:
movie_popularity.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3666 entries, 0 to 3684
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   movieId             3666 non-null   object
 1   popularity_ceiling  3666 non-null   object
dtypes: object(2)
memory usage: 85.9+ KB


In [36]:
movie_popularity["popularity_ceiling"] = movie_popularity["popularity_ceiling"]+"$$popularity"

In [37]:
print(Recommender_Graph_v2.number_of_nodes())
print(Recommender_Graph_v2.number_of_edges())

9945
1019660


In [38]:
movie_popularity['combined']= movie_popularity.values.tolist()
movie_popularity["combined_tuple"]=movie_popularity["combined"].apply(lambda x: tuple(x))
Recommender_Graph_v2.add_edges_from(list(movie_popularity["combined_tuple"]),weight = 4)
nx.write_gpickle(Recommender_Graph_v2, "Recommender_graph_year+genre+prod_country+pop.gpickle")

In [39]:
print(Recommender_Graph_v2.number_of_nodes())
print(Recommender_Graph_v2.number_of_edges())

10045
1023326


In [47]:
production_companies = movie_v2[["movieId","production_companies"]].copy()
production_companies.shape
production_companies.dropna(axis=0,inplace=True)
production_companies["without_string"] = production_companies["production_companies"].apply(lambda x: eval(x))
production_companies["without_string"].isna().sum()

0

In [49]:
production_companies = production_companies.explode(column = "without_string",ignore_index=False)
production_companies.dropna(axis=0,inplace=True)

In [55]:
production_companies.reset_index(inplace=True)

In [61]:
production_companies["production_companies_name"] = production_companies["without_string"].apply(lambda x: x["name"])

In [63]:
production_companies = production_companies[["movieId","production_Companies_name"]]

In [65]:
production_companies["production_Companies_name"] = production_companies["production_Companies_name"] + "$$production_companies"

<ipython-input-65-69bc5ba18c61>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  production_companies["production_Companies_name"] = production_companies["production_Companies_name"] + "$$production_companies"


In [67]:
print(Recommender_Graph_v2.number_of_nodes())
print(Recommender_Graph_v2.number_of_edges())

10045
1023326


In [68]:
production_companies['combined']= production_companies.values.tolist()
production_companies["combined_tuple"]=production_companies["combined"].apply(lambda x: tuple(x))
Recommender_Graph_v2.add_edges_from(list(production_companies["combined_tuple"]),weight = 4)
nx.write_gpickle(Recommender_Graph_v2, "Recommender_graph_year+genre+prod_country+pop.gpickle")

<ipython-input-68-7140033c927d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  production_companies['combined']= production_companies.values.tolist()
<ipython-input-68-7140033c927d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  production_companies["combined_tuple"]=production_companies["combined"].apply(lambda x: tuple(x))


In [69]:
print(Recommender_Graph_v2.number_of_nodes())
print(Recommender_Graph_v2.number_of_edges())
#only about 3k increase in no of nodes is similar to the no of movies out there (there are about 3k Dis5tinct production company nodes)

13007
1031520


In [70]:
production_companies["production_Companies_name"].nunique()  #no of nodes match the increase in the no of nodes in the graph

2962

In [72]:
movie_v2[["release_date","year"]].head(2)  #release date and year look the same so not using release date

,release_date,year
0,1995-10-30,1995
1,1995-11-16,1995


In [78]:
movie_runtime = movie_v2[["runtime","movieId"]].copy()
movie_runtime["runtime"] = movie_runtime["runtime"].astype(str)
movie_runtime["runtime"] = movie_runtime["runtime"]+"$$runtime"

In [79]:
print(Recommender_Graph_v2.number_of_nodes())
print(Recommender_Graph_v2.number_of_edges())

13007
1031520


In [80]:
movie_runtime['combined']= movie_runtime.values.tolist()
movie_runtime["combined_tuple"]=movie_runtime["combined"].apply(lambda x: tuple(x))
Recommender_Graph_v2.add_edges_from(list(movie_runtime["combined_tuple"]),weight = 4)
nx.write_gpickle(Recommender_Graph_v2, "Recommender_graph_year+genre+prod_country+pop+runtime+prod_comp.gpickle")

In [81]:
print(Recommender_Graph_v2.number_of_nodes())
print(Recommender_Graph_v2.number_of_edges())

13170
1035205


In [10]:
#spoken languages as a node
movie_spoken_languages = movie_v2[["movieId","spoken_languages"]]

In [11]:
movie_spoken_languages["spoken_languages"]

0       [{'english_name': 'English', 'iso_639_1': 'en'...
1       [{'english_name': 'English', 'iso_639_1': 'en'...
2       [{'english_name': 'English', 'iso_639_1': 'en'...
3       [{'english_name': 'Spanish', 'iso_639_1': 'es'...
4                                                      []
                              ...                        
3680    [{'english_name': 'English', 'iso_639_1': 'en'...
3681    [{'english_name': 'English', 'iso_639_1': 'en'...
3682    [{'english_name': 'English', 'iso_639_1': 'en'...
3683    [{'english_name': 'English', 'iso_639_1': 'en'...
3684    [{'english_name': 'English', 'iso_639_1': 'en'...
Name: spoken_languages, Length: 3685, dtype: object

In [12]:
movie_spoken_languages = movie_v2[["movieId","spoken_languages"]].copy()
print(movie_spoken_languages.shape)
movie_spoken_languages.dropna(axis=0,inplace=True)
movie_spoken_languages["without_string"] = movie_spoken_languages["spoken_languages"].apply(lambda x: eval(x))
movie_spoken_languages["without_string"].isna().sum()

(3685, 2)


0

In [13]:
movie_spoken_languages.head(2)

,movieId,spoken_languages,without_string
0,1,"[{'english_name': 'English', 'iso_639_1': 'en'...","[{'english_name': 'English', 'iso_639_1': 'en'..."
1,10,"[{'english_name': 'English', 'iso_639_1': 'en'...","[{'english_name': 'English', 'iso_639_1': 'en'..."


In [14]:
movie_spoken_languages = movie_spoken_languages.explode(column = "without_string",ignore_index=False).reset_index()

In [15]:
print(movie_spoken_languages.shape)
movie_spoken_languages.dropna(axis=0,subset=["without_string"],inplace=True)
print(movie_spoken_languages.shape)

(5154, 4)
(5094, 4)


In [16]:
movie_spoken_languages["spoken_language_final"] = movie_spoken_languages["without_string"].apply(lambda x: x["english_name"])

In [17]:
movie_spoken_languages.head(2)

,index,movieId,spoken_languages,without_string,spoken_language_final
0,0,1,"[{'english_name': 'English', 'iso_639_1': 'en'...","{'english_name': 'English', 'iso_639_1': 'en',...",English
1,1,10,"[{'english_name': 'English', 'iso_639_1': 'en'...","{'english_name': 'English', 'iso_639_1': 'en',...",English


In [21]:
movie_spoken_languages["movieId"] = movie_spoken_languages["movieId"].astype(str)
movie_spoken_languages["movieId"] = movie_spoken_languages["movieId"] +"$$m"

In [22]:
movie_spoken_languages["spoken_language_final"] = movie_spoken_languages["spoken_language_final"] + "$$spoken_language"

In [23]:
movie_spoken_languages.head(2)

,index,movieId,spoken_languages,without_string,spoken_language_final
0,0,1$$m,"[{'english_name': 'English', 'iso_639_1': 'en'...","{'english_name': 'English', 'iso_639_1': 'en',...",English$$spoken_language
1,1,10$$m,"[{'english_name': 'English', 'iso_639_1': 'en'...","{'english_name': 'English', 'iso_639_1': 'en',...",English$$spoken_language


In [111]:
print(Recommender_Graph_v2.number_of_nodes())
print(Recommender_Graph_v2.number_of_edges())

13170
1035205


In [24]:
movie_spoken_languages = movie_spoken_languages[["movieId","spoken_language_final"]].copy()

movie_spoken_languages['combined']= movie_spoken_languages.values.tolist()

In [26]:
movie_spoken_languages["combined_tuple"]=movie_spoken_languages["combined"].apply(lambda x: tuple(x))
Recommender_Graph_v2.add_edges_from(list(movie_spoken_languages["combined_tuple"]),weight = 4)
nx.write_gpickle(Recommender_Graph_v2, "Recommender_graph_year+genre+prod_country+pop+runtime+prod_comp+spoken_lang.gpickle")

In [27]:
print(Recommender_Graph_v2.number_of_nodes())
print(Recommender_Graph_v2.number_of_edges())

13252
1040299


In [28]:
#check
movie_spoken_languages["spoken_language_final"].nunique()

82

In [ ]:
#user related attributes remaining and movie related only vote count and average is not used

In [4]:
Recommender_Graph_v2 = nx.read_gpickle("Recommender_graph_year+genre+prod_country+pop+runtime+prod_comp+spoken_lang.gpickle")

In [6]:
Recommender_Graph_v2.has_node("English$$spoken_language")

True

In [20]:
movie_v2.info()
#NOTES ON Movie Based Nodes
#based on eda remove the following columns
#9 adult - NO (because no movie is adult)
#10 backdrop_path - NO
#11 belongs_to_collection ,14 homepage  - NO (v less info available)
#genres.1 - NO (same info as genres)
#24 release date is not used because maybe the year includes that effect
#OVERVIEW is not used in the recommender graph

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3685 entries, 0 to 3684
Data columns (total 34 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   movieId                3685 non-null   int64  
 1   title                  3685 non-null   object 
 2   genres                 3685 non-null   object 
 3   year                   3685 non-null   int64  
 4   meta_data              3685 non-null   object 
 5   distinct_user_Count    3685 non-null   int64  
 6   count_of_ratings       3685 non-null   int64  
 7   sum_ratings            3685 non-null   int64  
 8   average_rating         3685 non-null   float64
 9   adult                  3666 non-null   object 
 10  backdrop_path          3405 non-null   object 
 11  belongs_to_collection  708 non-null    object 
 12  budget                 3666 non-null   float64
 13  genres.1               3666 non-null   object 
 14  homepage               323 non-null    object 
 15  id  

In [ ]:
#Adding user Database nodes

In [19]:
users = pd.read_csv("mv-1m_user_rf.csv")

In [14]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6040 entries, 0 to 6039
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   userId      6040 non-null   int64 
 1   gender      6040 non-null   object
 2   age         6040 non-null   int64 
 3   occupation  6040 non-null   object
 4   zipcode     6040 non-null   object
dtypes: int64(2), object(3)
memory usage: 236.1+ KB


In [15]:
users["userId"] =users["userId"].astype(str)
users["userId"] = users["userId"] +"$$u"
users.to_csv('user_modified_$$u.csv',index=False)

In [16]:
users["userId"] =users["userId"].astype(str)
users["userId"] = users["userId"] +"$$u"
users.to_csv('user_modified_$$u.csv',index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6040 entries, 0 to 6039
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   userId      6040 non-null   object
 1   gender      6040 non-null   object
 2   age         6040 non-null   int64 
 3   occupation  6040 non-null   object
 4   zipcode     6040 non-null   object
dtypes: int64(1), object(4)
memory usage: 236.1+ KB


In [19]:
users["age"] =users["age"].astype(str)
users["age"] = users["age"] +"$$age_user"

In [23]:
users["occupation"] = users["occupation"] +"$$occupation"

In [26]:
users["zipcode"] =users["zipcode"].astype(str)
users["zipcode"] = users["zipcode"] +"$$Zip_code_user"

In [28]:
users["gender"] = users["gender"] +"$$gender"

In [ ]:
#making user_features related nodes

In [37]:
users_1 = users[["userId","gender"]].copy()
users_1['combined']= users_1.values.tolist()
users_1["combined_tuple"]=users_1["combined"].apply(lambda x: tuple(x))
Recommender_Graph_v2.add_edges_from(list(users_1["combined_tuple"]),weight = 4)

In [34]:
Recommender_Graph_v2.number_of_nodes()

13254

In [38]:
nx.write_gpickle(Recommender_Graph_v2, "Recommender_graph_year+genre+prod_country+pop+runtime+prod_comp+spoken_lang+gender.gpickle")

In [39]:
Recommender_Graph_v2.number_of_nodes()

13254

In [40]:
users_1 = users[["userId","zipcode"]].copy()
users_1['combined']= users_1.values.tolist()
users_1["combined_tuple"]=users_1["combined"].apply(lambda x: tuple(x))
Recommender_Graph_v2.add_edges_from(list(users_1["combined_tuple"]),weight = 4)




users_1 = users[["userId","age"]].copy()
users_1['combined']= users_1.values.tolist()
users_1["combined_tuple"]=users_1["combined"].apply(lambda x: tuple(x))
Recommender_Graph_v2.add_edges_from(list(users_1["combined_tuple"]),weight = 4)

users_1 = users[["userId","occupation"]].copy()
users_1['combined']= users_1.values.tolist()
users_1["combined_tuple"]=users_1["combined"].apply(lambda x: tuple(x))
Recommender_Graph_v2.add_edges_from(list(users_1["combined_tuple"]),weight = 4)

In [41]:
Recommender_Graph_v2.number_of_nodes()

16721

In [42]:
users.head(2)

,userId,gender,age,occupation,zipcode
0,1$$u,F$$gender,1$$age_user,K-12 student$$occupation,48067$$Zip_code_user
1,2$$u,M$$gender,56$$age_user,self-employed$$occupation,70072$$Zip_code_user


In [44]:
13254 + users["age"].nunique()  + users["occupation"].nunique() +  users["zipcode"].nunique()

16721

In [ ]:
#reco'd

In [45]:
nx.write_gpickle(Recommender_Graph_v2, "Recommender_graph_year+genre+prod_country+pop+runtime+prod_comp+spoken_lang+gender+user_features.gpickle")

In [5]:
Recommender_Graph_v2 = nx.read_gpickle("Recommender_graph_year+genre+prod_country+pop+runtime+prod_comp+spoken_lang+gender+user_features.gpickle")

In [6]:
Recommender_Graph_v2.number_of_nodes()

16721

In [7]:
# break of complete graph into training and test graph
import copy

In [8]:
#use deepcopy instead of just copy (just copy does not copies graph correctly)
Train_Graph = copy.deepcopy(Recommender_Graph_v2) 

In [9]:
Train_Graph.has_edge("1$$u","2762$$m")

True

## Training the Knowledge Graph primarily by Page Rank Algorithm

In [10]:
print(Recommender_Graph_v2.number_of_nodes())

16721


In [11]:
print(Train_Graph.number_of_nodes())

16721


In [12]:
print(Recommender_Graph_v2.number_of_edges())

1064459


In [13]:
test_indices = pd.read_csv("Test_indices.csv")

In [14]:
#in the 5th coloumn there is an NA value
test_indices.head(4)

,0,1,2,3,4
0,2,0,8,13,7.0
1,5,1,15,25,10.0
2,23,3,17,58,11.0
3,30,4,20,59,12.0


In [15]:
user_movie_test = ratings.iloc[test_indices["0"]][["userId","movieId"]] #change the indices to 0,1,2,3,4 (to have the set for test dataset)

In [16]:
user_movie_test[["userId","movieId"]] = user_movie_test[['userId', 'movieId']].astype(str)

In [17]:
user_movie_test["userId"] = user_movie_test["userId"]+"$$u"  #u subscript for user
user_movie_test["movieId"] = user_movie_test["movieId"]+"$$m"  #u subscript for user

In [18]:
user_movie_test['combined']= user_movie_test.values.tolist()

In [19]:
user_movie_test["combined_tuple"]=user_movie_test["combined"].apply(lambda x: tuple(x))

In [106]:
#just an example
#e = ("1$$u","2762$$m")
#Train_Graph.remove_edge(e)

In [20]:
#removing the test indiced user and movie id edges from the Complete Knowledge Graphs
Train_Graph.remove_edges_from(list(user_movie_test["combined_tuple"]))

In [21]:
user_movie_test.shape

(200042, 4)

In [22]:
print(Train_Graph.number_of_edges())

864417


In [118]:
1064459 - 200042

864417

In [120]:
nx.write_gpickle(Train_Graph, "Train_Graph.gpickle")

In [ ]:
#weights avry karne hai , personalize karna hai leakage is not happening

In [55]:
print(Train_Graph.number_of_edges())

1064477


In [ ]:
graph - weights =4

In [ ]:
defining user

In [5]:
#the graph that we are using is an undirected graph
Train_Graph = nx.read_gpickle("Train_Graph.gpickle")

In [6]:
Recommender_Graph_v2 = nx.read_gpickle("Recommender_graph_year+genre+prod_country+pop+runtime+prod_comp+spoken_lang+gender+user_features.gpickle")

In [23]:
#tol = 1e-8 gives the no of iterations that took in the page rank algorithm (specifically adjacency matrix algorithm)
#pr = nx.pagerank(Train_Graph,alpha=.85,max_iter=6,tol=1e-5) -20.76
pr = nx.pagerank(Train_Graph,alpha=.85,max_iter=5,tol=1e-5)

In [ ]:
#average rating (calculated from train data) - new user rating> then user liked the movie - 

In [53]:
#personalization feature of pagerank function actually controls the landing spot

In [ ]:
#multiprocessing check

In [ ]:
output

In [7]:
pr = nx.pagerank(Train_Graph,personalization={'1$$u':1})
#sorting the nodes in descending order
Top_n_nodes = sorted(pr,key =pr.get, reverse=True)
#removing nan nodes
Top_n_nodes_2 = [x for x in Top_n_nodes if not (type(x)==float)]
#selecting top 10 movies
substr = "$$m"
selections = [x for x in Top_n_nodes_2 if substr in x][:10]

In [8]:
selections

['260$$m',
 '2028$$m',
 '608$$m',
 '1$$m',
 '527$$m',
 '1197$$m',
 '1097$$m',
 '2355$$m',
 '3114$$m',
 '919$$m']

In [39]:
#so the idea 1st first we will shortlist no of movies in the validation dataset does the user likes (by subtracting the average movie rating in the training dataset) and then accordingly pick the top x no of movies from the ranked list and then check the recall or precision @K
#finding average user rating
ratings.head(1)

,userId,movieId,rating,title,genres,year,meta_data,gender,age,occupation,zipcode
0,1,1,5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995,"{'adult': False, 'backdrop_path': '/3Rfvhy1Nl6...",F,1,K-12 student,48067


In [24]:
ratings[["userId","movieId"]] = ratings[["userId","movieId"]].astype(str)

In [25]:
ratings["userId"] = ratings["userId"] +"$$u"
ratings["movieId"] = ratings["movieId"] +"$$m"

In [26]:
ratings = ratings[["userId","movieId","rating"]]

In [27]:
Train_Data = pd.read_csv("Train_indices.csv")

In [28]:
Train_Data.head(2)

,0,1,2,3,4
0,0.0,2.0,0.0,0.0,0.0
1,1.0,5.0,1.0,1.0,1.0


In [29]:
#selecting the first train dataset
#we can change the train dataset by changing 0 in the below code
train_data = ratings.iloc[Train_Data["0"].dropna()]

In [30]:
average_rating_users = train_data.groupby('userId').agg({"rating": "mean"})

In [31]:
average_rating_users.rename(columns = {'rating':'avg_rating'}, inplace = True)

In [32]:
average_rating_users.head(2)

,avg_rating
userId,
1$$u,4.148936
10$$u,4.125402


In [33]:
train_data_avg_rating = train_data.merge(average_rating_users,left_on=["userId"],right_on=["userId"],how="left")

In [34]:
print(train_data_avg_rating.shape)
print(train_data.shape)
#merge is correct

(800167, 4)
(800167, 3)


In [35]:
train_data_avg_rating.head(2)

,userId,movieId,rating,avg_rating
0,1$$u,1$$m,5,4.148936
1,1$$u,2692$$m,4,4.148936


In [36]:
def flag (rating,avg_rating):
    if (rating>=avg_rating):
        return 1
    else:
        return 0

In [37]:
train_data_avg_rating["Flag"] = train_data_avg_rating.apply(lambda x: flag(x["rating"], x["avg_rating"]), axis=1)

In [38]:
train_data_avg_rating.shape

(800167, 5)

In [39]:
Test_Data = pd.read_csv("Test_indices.csv")

In [40]:
Validation_Dataset = ratings.iloc[Test_Data["0"].dropna()]

In [41]:
Validation_data_avg_rating = Validation_Dataset.merge(average_rating_users,left_on=["userId"],right_on=["userId"],how="left")

In [42]:
#actually taking the training dataset and calculating the average rating from that
print(Validation_data_avg_rating.shape)
print(Validation_Dataset.shape)

(200042, 4)
(200042, 3)


In [43]:
Validation_data_avg_rating["Flag"] = Validation_data_avg_rating.apply(lambda x: flag(x["rating"], x["avg_rating"]), axis=1)

In [44]:
Validation_liked_train0 = Validation_data_avg_rating[Validation_data_avg_rating["Flag"]==1]

In [45]:
#approximately 50%
Validation_liked_train0.shape  

(109114, 5)

In [46]:
user_liked_movie_test = Validation_liked_train0.groupby("userId")["movieId"].nunique()

In [47]:
user_liked_movie_test0 = user_liked_movie_test.copy()

In [48]:
user_liked_movie_test0.to_csv('user_liked_movie_test0.csv',index=True)

In [49]:
user_liked_movie_test0_v2 = user_liked_movie_test0.reset_index()

In [50]:
user_list = list(user_liked_movie_test0_v2["userId"])

In [51]:
len(user_list)

5957

In [52]:
Validation_liked_train0.head(2)

,userId,movieId,rating,avg_rating,Flag
1,1$$u,2804$$m,5,4.148936,1
3,1$$u,1029$$m,5,4.148936,1


In [ ]:
#multi processing & multi threading
#5 page ranks in parallel
#random walks
#deadlocks
#concating + into one result
#MVP
#PAGE_rANK_scipy

In [ ]:
#the below code is taking more than necessary time even for 1 iteration it is taking more than 6 min (on the contrary only one iteration takes about 20 sec to complete)
#I think it is going in an infinite loop

import multiprocessing
from multiprocessing import Pool
def page_rank_optimize(a):
    return a**2
    #return nx.pagerank(Train_Graph,alpha,max_iter,tol)

if __name__ == '__main__':
    # Create a multiprocessing pool with four processes
    pool = multiprocessing.Pool(1)
    #alpha=.85
    #max_iter=5
    #tol=1e-5
    print(page_rank_optimize(10))
    results = pool.map(page_rank_optimize, [10])
    pool.close()
    print(results)

100


In [ ]:
import multiprocessing

def square(x):
    return x**2

if __name__ == '__main__':
    pool = multiprocessing.Pool(2)
    results = pool.starmap(square, ([5],[6]))
    print(results)

In [7]:
import multiprocessing
num_cores = multiprocessing.cpu_count()
print(num_cores)

8


In [7]:
nx.pagerank(Train_Graph,alpha=.85,max_iter=5,tol=1e-5)

{'1$$u': 3.4211007789463886e-05,
 '1$$m': 0.0006310607821673088,
 '2692$$m': 0.00034334851202170487,
 '2762$$m': 0.0007834973626240149,
 '2791$$m': 0.0005072742387191645,
 '2797$$m': 0.0004284565401513623,
 '2804$$m': 0.00040370282931101226,
 '2918$$m': 0.0004169106271656689,
 '3105$$m': 0.00021059737492317828,
 '3114$$m': 0.0005125441325315626,
 '3186$$m': 0.00015535140320072178,
 '3408$$m': 0.0004262083115462942,
 '48$$m': 0.000129905437470707,
 '527$$m': 0.0007340356269689099,
 '531$$m': 0.0001112522735031565,
 '588$$m': 0.0003921588635750235,
 '594$$m': 0.00023474658572970856,
 '595$$m': 0.000325261140843412,
 '608$$m': 0.000767088425546403,
 '661$$m': 0.0001758181035404566,
 '720$$m': 0.00013638645789794342,
 '745$$m': 0.00021670881980868743,
 '783$$m': 0.00012769508933011263,
 '919$$m': 0.0005187336040888838,
 '938$$m': 7.65847278909508e-05,
 '2687$$m': 0.00016426594325856167,
 '260$$m': 0.0009223783811642469,
 '914$$m': 0.00019448166221542533,
 '2355$$m': 0.0005342055752213466,


In [121]:
metric_df = pd.DataFrame(columns=["userId","metric","intersection_length","recommendations_length"])

In [73]:
import multiprocessing



if __name__ == '__main__':
    pool = multiprocessing.Pool()
    
    outputs = pool.map(calculate_pagerank, inputs)
    print(outputs)

In [74]:
#Think Tomorrow

def parallel_pagerank(G, num_processes=4):
    def calculate_pagerank(G, user):
        return nx.pagerank(G, personalization={user:1},alpha=.85,max_iter=5,tol=1e-5)
    
    with Pool(num_processes) as pool:
        # Run PageRank on each chunk of nodes in parallel
        results = pool.map(calculate_pagerank, nodes_chunks)
    pr = []
    for result in results:
        pr.append(result)

In [76]:
pool = Pool(processes=num_processes)

In [ ]:
for user in user_list:
    #the movie list liked by user in the test dataset
    movie_validation_liked = list(set(Validation_liked_train0[Validation_liked_train0["userId"]==user]["movieId"]))
    pr = nx.pagerank(Train_Graph,personalization={user:1})
    #sorting the nodes in descending order
    Top_n_nodes = sorted(pr,key =pr.get, reverse=True)
    #removing nan nodes
    Top_n_nodes_2 = [x for x in Top_n_nodes if not (type(x)==float)]
    #selecting top 10 movies
    substr = "$$m"
    #making the length of validation dataset and pagerank algorithm dataset same (so that our recall or precision is not bounded)
    selections = [x for x in Top_n_nodes_2 if substr in x][:len(movie_validation_liked)]
    if len(movie_validation_liked) != 0:
        metric = len(list(set(selections).intersection(movie_validation_liked))) / (len(movie_validation_liked))
        metric_df.loc[len(metric_df.index)] = [user,metric,len(list(set(selections).intersection(movie_validation_liked))),len(selections)] 
        print(metric_df.shape)
        ARHR  , Scaled Hit Rank (arhr*), (Use weights basis no of movies the user has seen the no of movies)

(1, 4)
(2, 4)
(3, 4)
(4, 4)
(5, 4)
(6, 4)
(7, 4)
(8, 4)
(9, 4)
(10, 4)
(11, 4)
(12, 4)
(13, 4)
(14, 4)
(15, 4)
(16, 4)
(17, 4)
(18, 4)
(19, 4)
(20, 4)
(21, 4)
(22, 4)
(23, 4)
(24, 4)
(25, 4)
(26, 4)
(27, 4)
(28, 4)
(29, 4)
(30, 4)
(31, 4)
(32, 4)
(33, 4)
(34, 4)
(35, 4)
(36, 4)
(37, 4)
(38, 4)
(39, 4)
(40, 4)
(41, 4)
(42, 4)
(43, 4)
(44, 4)
(45, 4)
(46, 4)
(47, 4)
(48, 4)
(49, 4)
(50, 4)
(51, 4)
(52, 4)
(53, 4)
(54, 4)
(55, 4)
(56, 4)
(57, 4)
(58, 4)
(59, 4)
(60, 4)
(61, 4)
(62, 4)
(63, 4)
(64, 4)
(65, 4)
(66, 4)
(67, 4)
(68, 4)
(69, 4)
(70, 4)
(71, 4)
(72, 4)
(73, 4)
(74, 4)
(75, 4)
(76, 4)
(77, 4)
(78, 4)
(79, 4)
(80, 4)
(81, 4)
(82, 4)
(83, 4)
(84, 4)
(85, 4)
(86, 4)
(87, 4)
(88, 4)
(89, 4)
(90, 4)
(91, 4)
(92, 4)
(93, 4)
(94, 4)
(95, 4)
(96, 4)
(97, 4)
(98, 4)
(99, 4)
(100, 4)
(101, 4)
(102, 4)
(103, 4)
(104, 4)
(105, 4)
(106, 4)
(107, 4)
(108, 4)
(109, 4)
(110, 4)
(111, 4)
(112, 4)
(113, 4)
(114, 4)
(115, 4)
(116, 4)
(117, 4)
(118, 4)
(119, 4)
(120, 4)
(121, 4)
(122, 4)
(123, 4)
(

(924, 4)
(925, 4)
(926, 4)
(927, 4)
(928, 4)
(929, 4)
(930, 4)
(931, 4)
(932, 4)
(933, 4)
(934, 4)
(935, 4)
(936, 4)
(937, 4)
(938, 4)
(939, 4)
(940, 4)
(941, 4)
(942, 4)
(943, 4)
(944, 4)
(945, 4)
(946, 4)
(947, 4)
(948, 4)
(949, 4)
(950, 4)
(951, 4)
(952, 4)
(953, 4)
(954, 4)
(955, 4)
(956, 4)
(957, 4)
(958, 4)
(959, 4)
(960, 4)
(961, 4)
(962, 4)
(963, 4)
(964, 4)
(965, 4)
(966, 4)
(967, 4)
(968, 4)
(969, 4)
(970, 4)
(971, 4)
(972, 4)
(973, 4)
(974, 4)
(975, 4)
(976, 4)
(977, 4)
(978, 4)
(979, 4)
(980, 4)
(981, 4)
(982, 4)
(983, 4)
(984, 4)
(985, 4)
(986, 4)
(987, 4)
(988, 4)
(989, 4)
(990, 4)
(991, 4)
(992, 4)
(993, 4)
(994, 4)
(995, 4)
(996, 4)
(997, 4)
(998, 4)
(999, 4)
(1000, 4)
(1001, 4)
(1002, 4)
(1003, 4)
(1004, 4)
(1005, 4)
(1006, 4)
(1007, 4)
(1008, 4)
(1009, 4)
(1010, 4)
(1011, 4)
(1012, 4)
(1013, 4)
(1014, 4)
(1015, 4)
(1016, 4)
(1017, 4)
(1018, 4)
(1019, 4)
(1020, 4)
(1021, 4)
(1022, 4)
(1023, 4)
(1024, 4)
(1025, 4)
(1026, 4)
(1027, 4)
(1028, 4)
(1029, 4)
(1030, 4)
(1031,

In [ ]:
#ideally recommendation should not be the ones which the user has already seen
#ask sir for this should we remove those recommendations that the user has already seen in its training history
metric_df.to_csv('metric_df_train0_knowledge_graph.csv',index=False)

In [ ]:
import multiprocessing

def worker(num):
    

if __name__ == '__main__':
    # Create a list of processes
    processes = []
    for i in range(5):
        p = multiprocessing.Process(target=worker, args=(i,))
        processes.append(p)

    # Start the processes
    for p in processes:
        p.start()

    # Wait for all processes to complete
    for p in processes:
        p.join()

In [ ]:
https://www1.se.cuhk.edu.hk/~swang/VLDBJ2019-PAFO.pdf

In [ ]:
#biased evd

In [ ]:
pr = nx.pagerank(Train_Graph)

In [7]:
#NOTES
# how to change user specific node weights (maybe nodes connected to that user might be helpful)
# personalization
# one observation since movies are connected to more nodes as compared to user nodes (movies has features nodes as well so generally top ranks are occupied by the movies only)

In [ ]:
#Page_Rank Implementation

In [13]:
import numpy as np

def pagerank(M, num_iterations: int = 0, d: float = 0.85):
    """PageRank algorithm with explicit number of iterations. Returns ranking of nodes (pages) in the adjacency matrix.

    Parameters
    ----------
    M : numpy array
        adjacency matrix where M_i,j represents the link from 'j' to 'i', such that for all 'j'
        sum(i, M_i,j) = 1
    num_iterations : int, optional
        number of iterations, by default 100
    d : float, optional
        damping factor, by default 0.85

    Returns
    -------
    numpy array
        a vector of ranks such that v_i is the i-th rank from [0, 1],
        v sums to 1

    """
    N = M.shape[1]
    v = np.ones(N) / N
    M_hat = (d * M + (1 - d) / N)
    for i in range(num_iterations):
        v = M_hat @ v
    return v

M = np.array([[0, 0, 0, 0, 1],
              [0.5, 0, 0, 0, 0],
              [0.5, 0, 0, 0, 0],
              [0, 1, 0.5, 0, 0],
              [0, 0, 0.5, 1, 0]])
v = pagerank(M, 100, 0.85)

In [14]:
v

array([0.25419178, 0.13803151, 0.13803151, 0.20599017, 0.26375504])

In [ ]:
#variables to be taken into Knowledge Graph
#17 original_language

In [4]:
#not even one movie is adult
movie_v2[movie_v2["adult"]==True].shape

(0, 34)

In [28]:
movie_v2["genres.1"][1]

"[{'id': 12, 'name': 'Adventure'}, {'id': 28, 'name': 'Action'}, {'id': 53, 'name': 'Thriller'}]"

In [33]:
movie_v2.iloc[1,:].head(4)

movieId                           10
title               GoldenEye (1995)
genres     Action|Adventure|Thriller
year                            1995
Name: 1, dtype: object

In [38]:
movie_v2["original_language"].value_counts()

en    3299
fr     123
it      43
de      39
es      33
ja      30
zh      18
cn      15
ru       9
fa       7
sv       7
nl       5
pt       4
da       4
no       3
sr       3
xx       2
ar       2
cs       2
hu       2
bn       2
pl       2
vi       1
ta       1
ca       1
bo       1
ka       1
sh       1
ko       1
tr       1
hi       1
mk       1
el       1
nb       1
Name: original_language, dtype: int64

In [1]:
#knowledge graph building


In [ ]:
ratings_original = ratings[["userId","movieId","rating"]].copy()